In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uom190346a/sleep-health-and-lifestyle-dataset")

print("Path to dataset files:", path)

#https://www.kaggle.com/code/jillanisofttech/sleep-health-and-lifestyle-predication-with-94-ac
#eksempel av noen sin løsning på dette datasetet

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(path+"/Sleep_health_and_lifestyle_dataset.csv")

In [ ]:
headAmount = 5 # for å skrive ut alle data punkt i csv file skriv data.size() istedenfor eit tall
data.head(headAmount)

In [ ]:
data.describe()

In [ ]:
print(data['Sleep Disorder'].unique())
print(data['Occupation'].unique())
print(data['BMI Category'].unique())

In [ ]:
data['Blood Pressure'].unique()

In [ ]:
# Vi deler opp blodtrykk siden det er to verdiar i ein kolonne
data_SBT = (pd.concat([data, data['Blood Pressure'].str.split('/', expand=True)], axis=1).
drop('Blood Pressure', axis=1))

In [ ]:
data_SBT

In [ ]:
data_SBT = data_SBT.rename(columns={0: 'Systolic Blood Pressure', 1: 'Diastolic Blood Pressure'})

In [ ]:
data_SBT